In [2]:
import spacy
import pandas as pd
from nltk import edit_distance
import tqdm

2023-10-04 10:21:21.398652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_excel("ita_deriv_9.xlsx")
df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,fqTot,gramCat,lemma,word,Phones,morph,...,actual_lemma_pos,extra,actual_lemma_phones,spacy_lemma,nltk_lemma,derivational,process,reason,spacy_lemma_2,spacy_pos_2
0,0,0.0,0.0,0.0,50518.0,P,a,a,a,NaN,...,P,NaN,a,a,a,NaN,NaN,NaN,a,X
1,1,1.0,1.0,1.0,5219.0,P IN P@,a,a,a,NaN,...,P,NaN,a,a,a,NaN,NaN,NaN,a,X
2,2,2.0,2.0,2.0,1544.0,P IN B@,a,a,a,NaN,...,P,NaN,a,a,a,NaN,NaN,NaN,a,NOUN
3,3,3.0,3.0,3.0,272.0,P IN C@,a,a,a,NaN,...,P,NaN,a,a,a,NaN,NaN,NaN,a,X
4,4,4.0,4.0,4.0,234.0,S,a,a,a,NaN,...,P,NaN,a,a,a,NaN,NaN,NaN,a,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73818,73818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73819,73819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73820,73820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73821,73821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df['nltk_extra'] = ''
for idx, row in tqdm.tqdm(df.iterrows()):
    w = row["word"]
    l = row["nltk_lemma"]
    try:
        df.loc[idx, 'nltk_extra'] = w[len(l):]
    except:
        print(w, l)

44002it [00:04, 10276.34it/s]

nana nan
nane nan
nani nan
nani nan
nano nan
nulla nan
nulla nan
nulla nan
nulla nan
nulla nan
nulla nan
nulla nan
nulla nan
nulle nan
nulli nan
nullo nan


73823it [00:07, 9854.86it/s] 

nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan


In [31]:
# gcs = dict(zip(zip(df.index, df["word"]), df["gramCat"]))
import collections
gcs = collections.defaultdict(list)
accs = collections.defaultdict(int)

for idx, row in tqdm.tqdm(df.iterrows()):
    gcs[row["word"]].append(row["gramCat"])

73823it [00:02, 28077.70it/s]


In [50]:
spacify = {
    'B': 'ADV',
    'E': 'NOUN',
    'G': 'ADJ',
    'V': 'VERB',
    'I': 'INTJ',
    'N': 'PRON',
    'S': 'NOUN'
}

from spacy.language import Language

seen = {}

@Language.component("gtagger")
def custom_tagger(doc):
    # Modify POS tags or other annotations as needed
    for token in doc:
        # Modify token attributes as required
        t = doc.text
        
        if doc in seen:
            token.pos_ = seen[doc]
            token.tag_ = ''
        
        if gc := gcs.get(t):
            try:
                token.pos_ = spacify.get(gc[accs[t] % len(gc)], 'X')
                token.tag_ = ''
                accs[t] += 1
                seen[doc] = token.pos_
            except:
                print(t, accs[t])
        else:
            print(token.text)
            input()
            token.pos_ = 'X'
        return doc

nlp = spacy.load("it_core_news_lg", disable=['parser', 'ner', 'tagger'])
# nlp.initialize()
nlp.add_pipe('gtagger', name='gtagger', before='lemmatizer')


<function __main__.custom_tagger(doc)>

In [51]:
df['spacy_lemma_2'] = ''
df['spacy_pos_2'] = ''
for idx, row in tqdm.tqdm(df.iterrows()):
    w = nlp(row["word"])[0]
    df.loc[idx, 'spacy_pos_2'] = w.pos_
    df.loc[idx, 'spacy_lemma_2'] = nlp(row["word"])[0].lemma_

821it [00:03, 254.21it/s]


KeyboardInterrupt: 

In [52]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,fqTot,gramCat,lemma,word,Phones,morph,pos,...,actual_lemma_pos,extra,actual_lemma_phones,spacy_lemma,nltk_lemma,derivational,process,reason,spacy_lemma_2,spacy_pos_2
0,0.0,0.0,0.0,50518.0,P,a,a,a,NaN,ADP,...,P,NaN,a,a,a,NaN,NaN,NaN,a,X
1,1.0,1.0,1.0,5219.0,P IN P@,a,a,a,NaN,ADP,...,P,NaN,a,a,a,NaN,NaN,NaN,a,X
2,2.0,2.0,2.0,1544.0,P IN B@,a,a,a,NaN,ADP,...,P,NaN,a,a,a,NaN,NaN,NaN,a,NOUN
3,3.0,3.0,3.0,272.0,P IN C@,a,a,a,NaN,ADP,...,P,NaN,a,a,a,NaN,NaN,NaN,a,X
4,4.0,4.0,4.0,234.0,S,a,a,a,NaN,ADP,...,P,NaN,a,a,a,NaN,NaN,NaN,a,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
73819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
73820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
73821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,


In [12]:
# df = pd.read_excel("ita_deriv_6.xlsx")

# suffixes = ['eggi', 'ific', 'izz', 'aggi', 'aggin', 'ai', 'aiol', 'anz', 'enz', 'eri', 'ezz', 'i\'', 'ier', 'ifici', 'in', 'ism', 'esim', 'ist', 'ita\'', 'eta\'', 'ta\'', 'ituden', 'etudin', 'izi', 'uzi', 'ment', 'toi', 'tor', 'sor', 'tric', 'tori', 'ur', 'zion', 'sion', 'ace', 'al', 'ar', 'ari', 'atori', 'bil', 'es', 'an', 'in', 'asc', 'esc', 'evol', 'ic', 'ier', 'ifer', 'ific', 'efic', 'il', 'iv', 'izi', 'os', 'str']

suffixes = {
    "E V event/process": ["izzare", "ificare"],
    "E E pertaining to/follwoing": ["ismo", "esimo"],
    "E E actively involved in": ["ista"],
    "E E quantity of": ["aglia", "ame", "ume", "ata"],
    "E E hit": ["ata"],
    "E E place with": ["eta", "eto"],
    "E E place for": ["ile"],
    "E E containing": ["ato"],
    "E E feminine": ["essa"],
    "E G related to": ["ale", "torio", "ino", "eo", "are", "ano", 
                            "esco", "aso", "ico", "istico", "ivo", 
                            "izio", "'ile", "oso"],
    "E G similar to": ["aceo", "ino", "oide", "igno", "are"],
    "E G belonging to": ["ardo", "asco", "ico", "ano"],
    "E G inhabitant of": ["ano", "ino", "etano", "itano", "aneo", "eo", "ese", "ate", 
                         "ota", "are", "esco", "ico", "eno"],
    "V N effect of": ["ata", "ione", "menti", "aggi", "ura", "nza", "io", "eria"],
    "V N state of": ["ato", "nza", "ia", "io", "ore"],
    "V N related to": ["toio", "toia", "torio"],
    "V N the point of": ["ando"],
    "V N agent of": ["tore", "trice", "iere", "iero", "ante", "aro"],
    "V G referring to": ["evole", "ivo", "torio", "ario"],
    "V V grade of": ["ellare", "arellare", "erellare", "icchiare", "acchiare", "ettare", "ellar", "ettar"],
    "G N quality of": ["ita'", "udine", "ezza", "izia", "igia", "aggine", "ia"],
    "G V change status of": ["izzare", "ificare"],
    "G N phenomenon of": ["ismo", "esimo"],
    "G N actively involved in": ["ista"],
    "G G manner of": []
}

# df['actual_lemma'] = ''
df['derivational'] = ''
df['process'] = ''
df['reason'] = ''

for idx, row in tqdm.tqdm(df.iterrows()):
    w = row['word']
    try:
        for typ, suffs in suffixes.items():
            for s in suffs:
                if s in row['nltk_extra']:
#                     goal_lemma = w[:w.index(s)]
#                     if lemma := find_closest_lemma(goal_lemma):
        #                 print(goal_lemma, lemma)
        #                 input()
#                         df.loc[idx, 'actual_lemma'] = lemma
#                         df.loc[idx, 'derivational'] = s
                    t1, t2, *reason = typ.split()
                    if row["gramCat"] == t2: # todo use t1
                        df.loc[idx, 'process'] = f"{t1} => {t2}"
                        df.loc[idx, 'reason'] = ' '.join(reason)
#                     break
#         for p in prefixes:
#             ...
    except Exception as e:
        print(e, idx, w) #, row)

73823it [00:21, 3399.88it/s]


In [13]:
df.to_excel("ita_deriv.xlsx")

In [8]:
df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,fqTot,gramCat,lemma,word,Phones,morph,...,extra,actual_lemma_phones,spacy_lemma,nltk_lemma,derivational,process,reason,spacy_lemma_2,spacy_pos_2,nltk_extra
0,0,0.0,0.0,0.0,50518.0,P,a,a,a,NaN,...,NaN,a,a,a,,,,a,X,
1,1,1.0,1.0,1.0,5219.0,P IN P@,a,a,a,NaN,...,NaN,a,a,a,,,,a,X,
2,2,2.0,2.0,2.0,1544.0,P IN B@,a,a,a,NaN,...,NaN,a,a,a,,,,a,NOUN,
3,3,3.0,3.0,3.0,272.0,P IN C@,a,a,a,NaN,...,NaN,a,a,a,,,,a,X,
4,4,4.0,4.0,4.0,234.0,S,a,a,a,NaN,...,NaN,a,a,a,,,,a,X,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73818,73818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,,,,NaN,NaN,
73819,73819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,,,,NaN,NaN,
73820,73820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,,,,NaN,NaN,
73821,73821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,,,,NaN,NaN,
